In [ ]:
from argparse import ArgumentParser
import numpy as np
import pandas as pd
import torch
import os
import matplotlib.pyplot as plt
from Datamodule  import DataModule
from pytorch_lightning import Trainer
from load_model import load_model

results_dir='Results'
datapath='data'

# data params
train_res='128x128'
train_energy='all'

test_res='128x128'
test_energy='193'

max_samples=-1
batch_size=4
cached=False

# training params
criterion='sq_err'
lr=1e-3
amsgrad=True

# Model Params
model_type='UNET'
n_layers=4
hidden_dim=16
kernel_size=5

saved=True


In [ ]:
def visualize_target_output(pred, y):
    pred, y = pred.mean(dim=0).detach().numpy(), y.mean(dim=0).detach().numpy()

    minmin = torch.min(torch.tensor(
        [pred.min().item(), y.min().item()]
        ))

    maxmax = torch.max(torch.tensor(
        [pred.max().item(), y.max().item()]
        ))

    fig, axs = plt.subplots(nrows=1, ncols=2, sharey=True, sharex=True)
    im = axs[0].imshow(pred, vmin=minmin, vmax=maxmax, cmap='bone')
    im = axs[1].imshow(y, vmin=minmin, vmax=maxmax, cmap='bone')

    axs[0].set_title('Prediction')
    axs[1].set_title('Target')
    fig.tight_layout()

    fig.subplots_adjust(right=0.85)
    cbar_ax = fig.add_axes([0.88, 0.15, 0.04, 0.7])
    fig.colorbar(im, cax=cbar_ax)
    # plt.tight_layout()
    plt.show()

dm_trained = DataModule(
    datapath=datapath,
    cached=cached,
    max_samples=max_samples,
    batch_size=batch_size,
    num_workers=1,
    res=train_res,
    energy=train_energy
    )
model, model_name = load_model(
    model_type=model_type,
    h_dim=hidden_dim, 
    n_layers=n_layers,
    k_size=kernel_size,
    dm=dm_trained,
    saved=True,
    results_dir=results_dir,
    train_energy='all',
    train_res=train_res
    )
del dm_trained
dm_test = DataModule(
    datapath=datapath,
    cached=cached,
    max_samples=max_samples,
    batch_size=batch_size,
    num_workers=1,
    res=train_res,
    energy=train_energy)
dm_test.setup()
dl_test = dm_test.test_dataloader()

# count = 0
for batch_idx, batch in enumerate(dl_test):
    abc, fns = batch
    ab = abc[:, :32]
    pred, y = model.inference_step(batch, batch_idx, double_data_by_sym=False)
    visualize_target_output(pred[0], y[0])
    # count+=1
    # if count > 1:
    break

